In [149]:
import pandas as pd
import json 
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

In [150]:
nodes = json.load(open('./node_list.json', 'rb'))
edges = json.load(open('./edge_list.json', 'rb'))

In [151]:
DirectG = nx.DiGraph()

In [152]:
[DirectG.add_node(n["id"], nation=n["nation"]) for n in nodes]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [145]:
[DirectG.add_edge(edge['From'], edge['to'], type=edge['type']) for edge in edges];

In [146]:
#NODE Colors
color_mapping= {
    "Mondstadt" : "#aff8db",
    "Liyue": "#ffcbc1",
    "Inazuma" : "#d5aaff",
    "Snezhnaya": "#ace7ff", 
    "Sumeru" : "#175721",
    "Fontaine" : "#008CFF"
}

In [147]:
node_colors = [color_mapping.get(n[1].get("nation", "#d1d1d1")) for n in list(DirectG.nodes(data=True))]

In [148]:
node_colors= [color_mapping[n[1]["nation"]] if n[1]["nation"] in color_mapping.keys() else "#d1d1d1" for n in list(DirectG.nodes(data=True))]

KeyError: 'nation'

In [134]:
# position, so nodes in draw take on same positions, generally
pos = nx.spring_layout(DirectG, k=0.25, iterations=25)